![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
// We need to load assemblies at the start in their own cell
#load "../Initialize.csx"
#load "../QuantConnect.csx"

using System;
using System.Linq;
using System.Collections.Generic;

using QuantConnect;
using QuantConnect.Data.Market;
using QuantConnect.Research;

In [ ]:
using System;
using System.Linq;
using System.Collections.Generic;
using Newtonsoft.Json;

// =====================================
// Helpers
// =====================================

decimal Round2(decimal value) => Math.Round(value, 2);

decimal Median(List<decimal> values)
{
    if (values == null || values.Count == 0) return 0m;
    var sorted = values.OrderBy(x => x).ToList();
    int n = sorted.Count;
    int mid = n / 2;
    return (n % 2 == 0)
        ? (sorted[mid - 1] + sorted[mid]) / 2m
        : sorted[mid];
}

decimal StdDev(List<decimal> values)
{
    if (values == null || values.Count == 0) return 0m;
    var mean = values.Average();
    var sumSq = values.Sum(v => (v - mean) * (v - mean));
    return (decimal)Math.Sqrt((double)(sumSq / values.Count));
}

// =====================================
// DTO for JSON output (camelCase)
// =====================================

public class WeeklyBarRecord
{
    public string startDate { get; set; }
    public string endDate { get; set; }
    public decimal open { get; set; }
    public decimal high { get; set; }
    public decimal low { get; set; }
    public decimal close { get; set; }
    public decimal volume { get; set; }
    public int tradingDays { get; set; }
    public decimal openToHigh { get; set; }
    public decimal openToLow { get; set; }
    public decimal openToClose { get; set; }
    public decimal totalDelta { get; set; }
}

// =====================================
// QuantBook setup
// =====================================

var qb = new QuantBook();

const int MinTradingDays = 4;
const string TickerSymbol = "QQQ";

var HistoryStartDate = new DateTime(2010, 1, 1);
var HistoryEndDate   = new DateTime(2025, 12, 1);

// Load data
var equity = qb.AddEquity(TickerSymbol, Resolution.Daily);
equity.SetDataNormalizationMode(DataNormalizationMode.SplitAdjusted);

var dailyBars = qb.History<TradeBar>(
    equity.Symbol,
    HistoryStartDate,
    HistoryEndDate,
    Resolution.Daily
)
.OrderBy(b => b.EndTime)
.ToList();

// =====================================
// Group into weekly bars
// =====================================

var weeklyBars = new List<TradeBar>();
var weeklyTradingDates = new Dictionary<DateTime, List<DateTime>>();

var groupedByWeek = dailyBars
    .GroupBy(bar =>
    {
        int dow = (int)bar.Time.DayOfWeek;
        int mondayOffset = dow == 0 ? -6 : 1 - dow;
        return bar.Time.Date.AddDays(mondayOffset);
    })
    .OrderBy(g => g.Key);

foreach (var group in groupedByWeek)
{
    var weekBars = group.OrderBy(x => x.Time).ToList();

    if (weekBars.Count < MinTradingDays)
        continue;

    var first = weekBars.First();
    var last  = weekBars.Last();

    var weekly = new TradeBar
    {
        Time   = first.Time,
        Symbol = first.Symbol,
        Open   = first.Open,
        High   = weekBars.Max(b => b.High),
        Low    = weekBars.Min(b => b.Low),
        Close  = last.Close,
        Volume = weekBars.Sum(b => b.Volume)
    };

    weeklyBars.Add(weekly);
    weeklyTradingDates[weekly.Time] =
        weekBars.Select(b => b.Time.Date).ToList();
}

// =====================================
// Build weekly JSON records
// =====================================

var weeklyRecords = new List<WeeklyBarRecord>();

foreach (var bar in weeklyBars)
{
    decimal open  = bar.Open;
    decimal high  = bar.High;
    decimal low   = bar.Low;
    decimal close = bar.Close;

    // Percentages
    decimal pctOpenToHigh  = open != 0 ? (high - open)  / open * 100m : 0m;
    decimal pctOpenToLow   = open != 0 ? (low  - open)  / open * 100m : 0m;
    decimal pctOpenToClose = open != 0 ? (close - open) / open * 100m : 0m;

    // NEW: totalDelta
    decimal pctTotalDelta =
        (close > open)
        ? ((high - low) / low * 100m)   // green candle (low → high)
        : ((high - low) / high * 100m); // red candle (high → low)

    weeklyTradingDates.TryGetValue(bar.Time, out var dates);
    var endDate = dates?.Last() ?? bar.Time.Date;

    weeklyRecords.Add(new WeeklyBarRecord
    {
        startDate    = bar.Time.Date.ToString("yyyy-MM-dd"),
        endDate      = endDate.ToString("yyyy-MM-dd"),
        open         = Round2(open),
        high         = Round2(high),
        low          = Round2(low),
        close        = Round2(close),
        volume       = Round2(bar.Volume),
        tradingDays  = dates?.Count ?? 0,
        openToHigh   = Round2(pctOpenToHigh),
        openToLow    = Round2(pctOpenToLow),
        openToClose  = Round2(pctOpenToClose),
        totalDelta   = Round2(pctTotalDelta)
    });
}

// =====================================
// Compute statistics
// =====================================

var openToHighValues  = weeklyRecords.Select(r => r.openToHigh).ToList();
var openToLowValues   = weeklyRecords.Select(r => r.openToLow).ToList();
var openToCloseValues = weeklyRecords.Select(r => r.openToClose).ToList();
var totalDeltaValues  = weeklyRecords.Select(r => r.totalDelta).ToList();

Console.WriteLine("Weekly Percentage Move Statistics (2-decimal rounded):\n");

// openToHigh
Console.WriteLine(
    $"openToHigh  -> mean: {Round2(openToHighValues.Average())}%, " +
    $"median: {Round2(Median(openToHighValues))}%, " +
    $"std dev: {Round2(StdDev(openToHighValues))}%");

// openToLow
Console.WriteLine(
    $"openToLow   -> mean: {Round2(openToLowValues.Average())}%, " +
    $"median: {Round2(Median(openToLowValues))}%, " +
    $"std dev: {Round2(StdDev(openToLowValues))}%");

// openToClose
Console.WriteLine(
    $"openToClose -> mean: {Round2(openToCloseValues.Average())}%, " +
    $"median: {Round2(Median(openToCloseValues))}%, " +
    $"std dev: {Round2(StdDev(openToCloseValues))}%");

// totalDelta
Console.WriteLine(
    $"totalDelta  -> mean: {Round2(totalDeltaValues.Average())}%, " +
    $"median: {Round2(Median(totalDeltaValues))}%, " +
    $"std dev: {Round2(StdDev(totalDeltaValues))}%");

Console.WriteLine();

// =====================================
// Final JSON object (top-level)
// =====================================

var finalObject = new
{
    ticker = TickerSymbol,
    analysisStartDate = HistoryStartDate.ToString("yyyy-MM-dd"),
    analysisEndDate = HistoryEndDate.ToString("yyyy-MM-dd"),
    minTradingDayWeekThreshold = MinTradingDays,
    weeklyData = weeklyRecords
};

// Output JSON
JsonConvert.SerializeObject(finalObject, Formatting.Indented)
